# Bayesian estimation of open loop TAR models


In [187]:
import numpy as np
import TAR
import pandas as pd
from scipy.special import gamma, factorial
from scipy.stats import bernoulli

In [202]:

# Generate TAR series:
y, s = TAR.tar_simulation([2,0.9], [-2,0.5], 50, sd = 0.5)
dat = pd.DataFrame({"y":[i[0] for i in y], "s_0":s})
dat['x'] = dat["y"].shift(1)
dat.dropna(inplace = True)
dat.drop("s_0", axis=1, inplace = True)
# dat

{'Regime_1': 24, 'Regime_2': 26}


In [441]:
# Inputs for Bayesian inference:
def Estimate(dat, p_success):
    
    ## Keep the original data clean by operating on its copy rather than on itself:
    data = dat.copy()
    

    # Define the sequence of state variables:
    data["s_0"] = [bernoulli.rvs(p_success) for i in range(len(data))]
    # data["s_0"] = s[1:]
    n_0 = np.sum(data.s_0)
    n = len(data)
 
    data["xs_0"] = data.x*data.s_0
    data["s_1"] = 1 - data.s_0
    data["xs_1"] = data["s_1"]*data["x"]
    data["ys_0"] = data.y*data.s_0
    data["ys_1"] = data.y*data.s_1
    
    Sxx_0, Sxx_1 = data.loc[:, ["s_0", "xs_0"]].T@data.loc[:, ["s_0", "xs_0"]], \
        data.loc[:, ["s_1", "xs_1"]].T@data.loc[:, ["s_1", "xs_1"]]
    Sxy_0, Sxy_1 = data.loc[:, ["s_0", "xs_0"]].T@data.ys_0, \
        data.loc[:, ["s_1", "xs_1"]].T@data.ys_1
    print(Sxx_0, Sxx_1, Sxy_0, Sxy_1)
    delta_0, delta_1 = data.ys_0.T@data.ys_0 - Sxy_0.T@np.linalg.inv(Sxx_0)@Sxy_0, \
        data.ys_1.T@data.ys_1 - Sxy_1.T@np.linalg.inv(Sxx_1)@Sxy_1
    return delta_0, delta_1, Sxx_0, Sxx_1, Sxy_0, Sxy_1, n_0, n, data

def Update(delta_0, delta_1, Sxx_0, Sxx_1, Sxy_0, Sxy_1, n_0, n, data):
    return (gamma((n_0-1-3)/2)*gamma((n - n_0 - 1 - 3)/2)*delta_0**(-(n_0 - 1 -1)/2)*delta_1**(-(n - n_0 - 1 -1)/2))/(np.linalg.det(Sxx_0.to_numpy()@Sxx_1.to_numpy())**0.5)
    

In [477]:
delta_0, delta_1, Sxx_0, Sxx_1, Sxy_0, Sxy_1, n_0, n, data = Estimate(dat, 0.8)
Update(delta_0, delta_1, Sxx_0, Sxx_1, Sxy_0, Sxy_1, n_0, n, data)

            s_0       xs_0
s_0   34.000000 -32.519509
xs_0 -32.519509  80.981529             s_1       xs_1
s_1   15.000000 -12.780587
xs_1 -12.780587  33.820470 s_0    -32.302057
xs_0    -1.766411
dtype: float64 s_1    -12.569999
xs_1    -9.466518
dtype: float64


7.146271108706122e-19

In [317]:
print(data.ys_1.T@data.ys_1 - Sxy_1@np.linalg.pinv(Sxx_1)@Sxy_1, delta_1)


5.041182840546881 5.041182840546881


In [322]:
delta_0**(11)

4396440.019723648

1.721210100075389e-13

In [334]:
gamma((n_0-4)/2)*gamma((n-n_0-4)/2)/((np.linalg.det(Sxx_0.to_numpy()@Sxx_1.to_numpy())**(0.5))*delta_0*delta_1)

False

(            s_0        xs_0
 s_0   15.000000  -40.090014
 xs_0 -40.090014  130.694008,
             s_1        xs_1
 s_1   34.000000   36.831539
 xs_1  36.831539  112.603060)

In [5]:
import pystan as stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

In [6]:
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

In [8]:
posterior = pystan.build(schools_code, data=schools_data, random_seed=1)

AttributeError: module 'pystan' has no attribute 'build'